In [3]:
from ultralytics import YOLO
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

path_to_data = "/Users/thomasmcconnell/Library/CloudStorage/OneDrive-PomonaCollege/School/CS153/Project/cs153-football-formation-id/sample_data"
output_json_path = "/Users/thomasmcconnell/Library/CloudStorage/OneDrive-PomonaCollege/School/CS153/Project/cs153-football-formation-id/yolo_player_detector/"

I will use this notebook to detect people on each image that has been pre processed and cropped in the field_lines_detection folder. I will load yolov8n.pt model using ultralytics and then feed that model output to the final classification model. Change any data references to your data as needed.

In [ ]:
# load YOLO
model = YOLO('yolov8n.pt')

all_detections = []

# grab all cropped_sideline images
cropped_image_paths = []
for formation_folder in os.listdir(path_to_data):
    formation_path = os.path.join(path_to_data, formation_folder)
    if not os.path.isdir(formation_path):
        continue

    for video_folder in os.listdir(formation_path):
        video_path = os.path.join(formation_path, video_folder)
        if not os.path.isdir(video_path):
            continue

        for file in os.listdir(video_path):
            if file.startswith("cropped_sideline") and file.endswith(".png"):
                full_path = os.path.join(video_path, file)
                cropped_image_paths.append(full_path)

# detect players
for img_path in tqdm(cropped_image_paths, desc="Running YOLO Detection"):
    results = model(img_path, conf=0.25)  # returns list
    result = results[0]  # get first result

    player_data = []
    for box in result.boxes.data.tolist():  # (x1, y1, x2, y2, confidence, class)
        x1, y1, x2, y2, conf, cls = box

        # Only save if it's a 'person' (person = 0 in a YOLOv8 model)
        if int(cls) == 0:
            player_data.append({
                "bbox": [x1, y1, x2, y2],
                "confidence": conf
            })

    # save detections
    all_detections.append({
        "image_filename": os.path.basename(img_path),
        "players": player_data
    })

# save to json
with open(output_json_path + "player_bboxes.json", "w") as f:
    json.dump(all_detections, f, indent=4);


In [ ]:
results = model(cropped_image_paths[0], conf=0.25)
results[0].show()